# 客户购买预测
https://www.kesci.com/home/competition/5c234c6626ba91002bfdfdd3/leaderboard

In [110]:
#读取csv格式文件
import pandas as pd
path = './input/' #用户的路径环境
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import lightgbm as lgb
train = pd.read_csv(path+'train_set.csv')
test = pd.read_csv(path+'test_set.csv')


In [111]:
test['y']=-1
data = pd.concat([train,test])

## 类别特征自身计算groupby组合条件下的count特征

In [112]:
def feature_count(data, features):
    feature_name = 'count'
    for i in features:
        feature_name += '_' + i
    temp = data.groupby(features).size().reset_index().rename(columns={0: feature_name})
    data = data.merge(temp, 'left', on=features)
    return data,feature_name
ll=[]
for f in['campaign', 'contact','default','education','housing','job','loan','marital','poutcome','pdays','previous']:
    data,_=feature_count(data,['day','month',f])
    ll.append(_)

In [113]:
def feat_count(df, df_feature,fe,value,name=""):
    df_count = pd.DataFrame(df_feature.groupby(fe)[value].count()).reset_index()
    if not name:
        df_count.columns = fe + [value+"_%s_count" % ("_".join(fe[0]))]
    else:
        df_count.columns = fe + [name]
    df = df.merge(df_count, on=fe, how="left")#.fillna(0)
    return df,df_count.columns[1]


## 类别和类别特征组合做count和unique

In [114]:
sparse_feature= ['campaign','contact','default','education','housing','job','loan','marital','month','poutcome']
dense_feature=['age', 'balance','day','duration','campaign','pdays','previous']
crossf=[]
for d in tqdm(sparse_feature):
    for s in sparse_feature:
        crossf+=['count_'+str(d)+'_'+str(s),'nunique_'+str(d)+'_'+str(s)]
        temp=data.groupby(d)[s].agg(['count','nunique']).reset_index().rename(columns={'count':'count_'+str(d)+'_'+str(s),'nunique':'nunique_'+str(d)+'_'+str(s)})
        data=pd.merge(data,temp,on=d,how='left')

100%|██████████| 10/10 [00:18<00:00,  1.86s/it]


## 类别和类别交叉

In [115]:
# for i in sparse_feature:
#     for ii in sparse_feature:
#         if i != ii:
#             data[i + '_' + ii] = data[i].astype('str') + data[ii].astype('str')

## 类别和数值特征组合做各种最大最小标准差等统计值

In [116]:
for d in tqdm(sparse_feature):
    for s in dense_feature:
        crossf+=[str(d)+'_mean_'+str(s),str(d)+'_max_'+str(s),str(d)+'_min_'+str(s),str(d)+'_std_'+str(s)]
        temp=data.groupby(d)[s].agg(['mean','max','min','std']).reset_index().rename(columns={'mean':str(d)+'_mean_'+str(s),'max':str(d)+'_max_'+str(s),'min':str(d)+'_min_'+str(s),'std':str(d)+'_std_'+str(s)})
        data=pd.merge(data,temp,on=d,how='left')

100%|██████████| 10/10 [03:09<00:00, 18.96s/it]


In [117]:
#ID这里好像没用'ID',
import numpy as np

from scipy.sparse import csr_matrix

from sklearn.model_selection import KFold
a= list(data)
a.remove('ID')
a.remove('y')
num_features= a


one_hot_feature =['job', 'marital','education','default','housing','loan','contact','month','poutcome']


data['new_con'] = data['job'].astype(str)
for i in ['marital', 'education', 'contact','month','poutcome']:
    data['new_con'] = data['new_con'].astype(str) + '_' + data[i].astype(str)
data['new_con'] = data['new_con'].apply(lambda x: ' '.join(x.split('_')))



In [118]:
for feature in one_hot_feature:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])

In [119]:
vector_feature = ['new_con']
train=data[data.y!=-1]
# train, test, train_y, test_y = train_test_split(train,train_y,test_size=0.2, random_state=2018)
test=data[data.y==-1]
res=test[['ID']]
# test=test.drop(['ID'],axis=1)
train_x=train[num_features]
test_x=test[num_features]
enc = OneHotEncoder()
for feature in one_hot_feature:
    enc.fit(data[feature].values.reshape(-1, 1))
    train_a=enc.transform(train[feature].values.reshape(-1, 1)).toarray()
    test_a = enc.transform(test[feature].values.reshape(-1, 1)).toarray()
    train_x= np.hstack((train_x, train_a))
    test_x = np.hstack((test_x, test_a))

In [120]:
cv=CountVectorizer()
for feature in vector_feature:
    cv.fit(data[feature])
    train_a = cv.transform(train[feature]).toarray()
    test_a = cv.transform(test[feature]).toarray()
    train_x = np.hstack((train_x, train_a))
    test_x = np.hstack((test_x, test_a))

# trainxx, vali_x, trainyy, vali_y = train_test_split(train_x,train_y,test_size=0.2, random_state=2018)
n_splits=5
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
clf = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=30, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary',metric= 'auc',
    subsample=0.95, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.02, random_state=2017
    )

train_y=train['y']
res['pred'] = 0
for train_idx, val_idx in kfold.split(train_x):
    clf.random_state = clf.random_state + 1
    train_x1 = train_x[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x[val_idx]
    test_y1 = train_y.loc[val_idx]
    #,(vali_x,vali_y)
    clf.fit(train_x1, train_y1,eval_set=[(train_x1, train_y1),(test_x1, test_y1)],eval_metric='auc',early_stopping_rounds=100)
    res['pred'] += clf.predict_proba(test_x)[:,1]

res['pred'] = res['pred']/5
res.to_csv('./basesubmission.csv', index=False)


[10]	training's auc: 0.941475	valid_1's auc: 0.925542
[11]	training's auc: 0.94152	valid_1's auc: 0.925509
[12]	training's auc: 0.942639	valid_1's auc: 0.926901
[13]	training's auc: 0.942878	valid_1's auc: 0.927116
[14]	training's auc: 0.943118	valid_1's auc: 0.927558
[15]	training's auc: 0.943211	valid_1's auc: 0.927593
[16]	training's auc: 0.943663	valid_1's auc: 0.927821
[17]	training's auc: 0.944272	valid_1's auc: 0.928381
[18]	training's auc: 0.944436	valid_1's auc: 0.929095
[19]	training's auc: 0.944758	valid_1's auc: 0.929158
[20]	training's auc: 0.944836	valid_1's auc: 0.929288
[21]	training's auc: 0.945045	valid_1's auc: 0.929295
[22]	training's auc: 0.945392	valid_1's auc: 0.929472
[23]	training's auc: 0.945596	valid_1's auc: 0.929761
[24]	training's auc: 0.945657	valid_1's auc: 0.929775
[25]	training's auc: 0.945774	valid_1's auc: 0.929654
[26]	training's auc: 0.945804	valid_1's auc: 0.929608
[27]	training's auc: 0.946044	valid_1's auc: 0.929731
[28]	training's auc: 0.94616